In [1]:
import pandas as pd
import numpy as np

In [2]:
%config Completer.use_jedi = False

In [13]:
# TODO: 26.04.21 use distinct preprocessing module (clean notebook)

In [3]:
columns = [
    "symboling",
    "normalized-losses",
    "make",
    "fuel-type",
    "aspiration",
    "num-of-doors",
    "body-style",
    "drive-wheels",
    "engine-location",
    "wheel-base",
    "length",
    "width",
    "height",
    "curb-weight",
    "engine-type",
    "num-of-cylinders",
    "engine-size",
    "fuel-system",
    "bore",
    "stroke",
    "compression-ratio",
    "horsepower",
    "peak-rpm",
    "city-mpg",
    "highway-mpg",
    "price",
]

In [4]:
df = pd.read_csv("data/imports-85.data", header=None, names=columns)

In [5]:
# Preprocessing (instructions)
# - '?' = missing values
# - ignore 'symboling'
# - Skip samples with missing values in 'normalized-losses' (target)

df = df.replace("?", np.nan)
df.drop("symboling", axis="columns", inplace=True)
df = df[df["normalized-losses"].notna()]

df.head(3)

,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
6,158,audi,gas,std,four,sedan,fwd,front,105.8,192.7,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710


In [6]:
len(df)

164

In [7]:
df.dtypes[:5]

normalized-losses    object
make                 object
fuel-type            object
aspiration           object
num-of-doors         object
dtype: object

In [8]:
# Correcting data types (also see: 'Attribute Information in `data/imports-85.names`')
numerics = [
    "normalized-losses",
    "wheel-base",
    "length",
    "width",
    "height",
    "curb-weight",
    "engine-size",
    "bore",
    "stroke",
    "compression-ratio",
    "horsepower",
    "peak-rpm",
    "city-mpg",
    "highway-mpg",
    "price"
]

objects = [
    "make",
    "fuel-type",
    "aspiration",
    "num-of-doors",
    "body-style",
    "drive-wheels",
    "engine-location", 
    "engine-type",
    "num-of-cylinders",
    "fuel-system",
]

print(len(numerics), len(objects))

15 10


In [9]:
# Since objects are already objects, we only need to assure 
# that continuous columns are interpreted correctly

for k, v in df.dtypes.items():
    if k in numerics and v == 'object':
        df[k] = df[k].astype(float)

In [10]:
# Cleaned DataFrame
df.head(3)

,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.4,10.0,102.0,5500.0,24,30,13950.0
4,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.4,8.0,115.0,5500.0,18,22,17450.0
6,158.0,audi,gas,std,four,sedan,fwd,front,105.8,192.7,...,136,mpfi,3.19,3.4,8.5,110.0,5500.0,19,25,17710.0


In [12]:
df.describe()

,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
count,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,160.000000,160.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000
mean,122.000000,98.164024,172.238415,65.598780,53.774390,2458.274390,117.957317,3.298437,3.237312,10.126951,96.207317,5137.804878,26.268293,31.847561,11466.518293
std,35.442168,5.120198,11.417833,1.923028,2.343942,475.087068,30.896294,0.267348,0.294210,3.836306,30.408563,479.459113,6.193305,6.514349,5803.490319
min,65.000000,86.600000,141.100000,60.300000,49.400000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,15.000000,18.000000,5118.000000
25%,94.000000,94.500000,165.675000,64.000000,52.000000,2090.750000,97.000000,3.050000,3.107500,8.700000,69.000000,4800.000000,22.000000,28.000000,7446.000000
50%,115.000000,96.550000,172.000000,65.400000,54.100000,2367.500000,109.000000,3.270000,3.270000,9.000000,91.000000,5200.000000,26.000000,32.000000,9268.500000
75%,150.000000,100.400000,177.800000,66.500000,55.500000,2785.500000,131.750000,3.550000,3.410000,9.400000,114.000000,5500.000000,31.000000,37.000000,14559.250000
max,256.000000,115.600000,202.600000,71.700000,59.800000,4066.000000,258.000000,3.940000,4.170000,23.000000,200.000000,6600.000000,49.000000,54.000000,35056.000000
